# Survey memory
This notebook provides sample [EDSL](https://docs.expectedparrot.com/) code demonstrating how to add "memories" of previous questions to other simulated survey questions, to allow comparison of responses to questions where memories of prior questions are not added.

EDSL is an open-source library for simulating surveys and experiments with AI. Please see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started.

## Benefits of simulated surveys
Some benefits of using AI agents to answer surveys are that agents can be presented any number of questions to answer - tirelessly! - and that questions can be presented with or without information about other responses that have already been given. This allows us to easily explore how agents respond to the same questions with different contexts.

## No-memory default
By default, EDSL survey questions are administered asynchronously to the agents and language models added to the survey when it is run. This means that questions are answered independently of each other, unless other rules have been specified. This default behavior is useful for speed and minimizing token usage (not repeating prior question contexts unnecessarily).

However, in some cases we may want an agent to have the context of some or all prior questions when answering a given question (e.g., to immitate a human's experience answering a survey). Or, we may simply want to explore the potential impact of prior question contexts by running new questions with and without memories of them.

## How it works
EDSL has several options for adding survey memories:

* `set_full_memory_mode()` adds a memory of all prior questions and answers to each new question
* `set_lagged_memory(n)` adds a memory of a specified number of prior questions and answers to each new question
* `add_targeted_memory(<new_question>, <prior_question>)` adds a memory of specific prior question to a specific new question
* `add_memory_collection(<new_question>, [<list_of_prior_questions>])` adds memories of a set of prior questions and answers to a specific new question

[Learn more about memory rules](https://docs.expectedparrot.com/en/latest/surveys.html#question-memory).

In [1]:
# pip install edsl

Installing the tools. Here we use free text and multiple choice questions; [learn more about other question types](https://docs.expectedparrot.com/en/latest/questions.html) to choose from:

In [2]:
from edsl.questions import QuestionFreeText, QuestionMultipleChoice
from edsl import Survey

Creating a survey of questions. Note that we include copies of questions in order for the results to show side-by-side responses where we have and have not added memories of prior questions:

In [3]:
q_credit = QuestionFreeText(
    question_name = "credit",
    question_text = """Describe the phenomenon of hearing about an idea and then 
    believing that you are responsible for it."""
)

q_ideas_with_credit = QuestionFreeText(
    question_name = "ideas_with_credit",
    question_text = "Give me an original idea for an applications for large language models."
)

q_ideas_no_credit = QuestionFreeText(
    question_name = "ideas_no_credit",
    question_text = "Give me an original idea for an applications for large language models."
)

q_original_with_credit = QuestionMultipleChoice(
    question_name = "original_with_credit",
    question_text = "The idea you just described - is it original?",
    question_options = ["Yes, I thought of it myself.",
                        "I may have heard about it before.",
                        "No, I have heard about it before.",
                        "I do not know."]
)

q_original_no_credit = QuestionMultipleChoice(
    question_name = "original_no_credit",
    question_text = "The idea you just described - is it original?",
    question_options = ["Yes, I thought of it myself.",
                        "I may have heard about it before.",
                        "No, I have heard about it before.",
                        "I do not know."]
)

Passing the questions to a `Survey`:

In [4]:
survey = Survey([q_credit,
                 q_ideas_with_credit,
                 q_ideas_no_credit,
                 q_original_with_credit,
                 q_original_no_credit])

Here we add memories so that the results will include (1) a response to the "original" question where the "idea" question included the context of the "credit" question and (2) a response to the "original" question where the "idea" question did *not* include the context of the "credit" question:

In [5]:
survey = (survey
          # Adding a memory of "credit" to the first iteration of "ideas" 
          .add_targeted_memory(q_ideas_with_credit, q_credit)
          # Adding memories of both "credit" and "ideas" to the first iteration of "original"
          .add_memory_collection(q_original_with_credit, [q_credit, q_ideas_with_credit])
          # Adding a memory of "ideas" where "credit" was not included to the second iteration of "original"
          .add_targeted_memory(q_original_no_credit, q_ideas_no_credit)
         )

In [6]:
results = survey.run()

In [7]:
results.select("ideas_with_credit", "original_with_credit", "ideas_no_credit", "original_no_credit").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ answer                     ┃ answer                    ┃ answer                     ┃ answer                    ┃
┃ .ideas_with_credit         ┃ .original_with_credit     ┃ .ideas_no_credit           ┃ .original_no_credit       ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ An original application    │ I may have heard about it │ One original application   │ Yes, I thought of it      │
│ for large language models  │ before.                   │ for large language models  │ myself.                   │
│ could be a 'Cultural       │                           │ could be a 'Cultural       │                           │
│ Exchange Facilitator' app. │                           │ Exchange Companion' app.   │                           │
│ This app would utilize the │                           │ This platform would use    │                           │
│ language model's ability   │                           │ the power of language      │                           │
│ to understand and generate │                           │ models to facilitate       │                           │
│ text in multiple languages │                           │ cross-cultural             │                           │
│ to help users learn about  │                           │ understanding and language │                           │
│ and understand different   │                           │ learning. Users could      │                           │
│ cultures. It could offer   │                           │ input their native         │                           │
│ interactive stories,       │                           │ language and cultural      │                           │
│ customs, idioms, and even  │                           │ context, and the app would │                           │
│ simulate conversations     │                           │ pair them with content and │                           │
│ with virtual characters    │                           │ conversations that are     │                           │
│ from diverse backgrounds.  │                           │ tailored to bridge gaps    │                           │
│ The app would aim to       │                           │ between different          │                           │
│ promote empathy and global │                           │ cultures. It would provide │                           │
│ understanding by providing │                           │ real-time translation,     │                           │
│ a deep, personalized       │                           │ cultural notes, and        │                           │
│ cultural exchange          │                           │ context explanations to    │                           │
│ experience.                │                           │ help users navigate        │                           │
│                            │                           │ conversations with people  │                           │
│                            │                           │ from other parts of the    │                           │
│                            │                           │ world. Additionally, it    │                           │
│                            │                           │ could include a feature to │                           │
│                            │                           │ connect users with         │                           │
│                            │                           │ language learning partners │                           │
│                            │                           │ looking to exchange        │                           │
│                            │                           │ knowledge in their         │                           │
│                            │                          

We can examine the question prompts to verify how the rules were applied. To see a list of all the components of the results that can be directly accessed:

In [8]:
results.columns

['agent.agent_instruction',
 'agent.agent_name',
 'answer.credit',
 'answer.ideas_no_credit',
 'answer.ideas_with_credit',
 'answer.original_no_credit',
 'answer.original_with_credit',
 'comment.original_no_credit_comment',
 'comment.original_with_credit_comment',
 'iteration.iteration',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_logprobs',
 'model.top_p',
 'prompt.credit_system_prompt',
 'prompt.credit_user_prompt',
 'prompt.ideas_no_credit_system_prompt',
 'prompt.ideas_no_credit_user_prompt',
 'prompt.ideas_with_credit_system_prompt',
 'prompt.ideas_with_credit_user_prompt',
 'prompt.original_no_credit_system_prompt',
 'prompt.original_no_credit_user_prompt',
 'prompt.original_with_credit_system_prompt',
 'prompt.original_with_credit_user_prompt',
 'question_options.credit_question_options',
 'question_options.ideas_no_credit_question_options',
 'question_options.ideas_with_credit_q

Here we inspect the system (agent) and user (question) prompts for the "credit" question:

In [9]:
results.select("credit_system_prompt", "credit_user_prompt").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ prompt                                                 ┃ prompt                                                 ┃
┃ .credit_system_prompt                                  ┃ .credit_user_prompt                                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ You are answering questions as if you were a human. Do │ You are being asked the following question: Describe   │
│ not break character. You are an agent with the         │ the phenomenon of hearing about an idea and then       │
│ following persona:                                     │     believing that you are responsible for it.         │
│ {}                                                     │ Return a valid JSON formatted like this:               │
│                                                        │ {"answer": "<put free text answer here>"}              │
└────────────────────────────────────────────────────────┴────────────────────────────────────────────────────────┘

Here we can verify that we included the context of the "credit" question in the "_with_memory" iteration of the "ideas" question:

In [10]:
results.select("ideas_with_credit_user_prompt", "original_with_credit_user_prompt").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ prompt                                                 ┃ prompt                                                 ┃
┃ .ideas_with_credit_user_prompt                         ┃ .original_with_credit_user_prompt                      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ You are being asked the following question: Give me an │ You are being asked the following question: The idea   │
│ original idea for an applications for large language   │ you just described - is it original?                   │
│ models.                                                │ The options are                                        │
│ Return a valid JSON formatted like this:               │                                                        │
│ {"answer": "<put free text answer here>"}              │ 0: Yes, I thought of it myself.                        │
│         Before the question you are now answering, you │                                                        │
│ already answered the following question(s):            │ 1: I may have heard about it before.                   │
│                 Question: Describe the phenomenon of   │                                                        │
│ hearing about an idea and then                         │ 2: No, I have heard about it before.                   │
│     believing that you are responsible for it.         │                                                        │
│         Answer: The phenomenon of hearing about an     │ 3: I do not know.                                      │
│ idea and then believing that you are responsible for   │                                                        │
│ it is often referred to as 'cryptomnesia.' It occurs   │ Return a valid JSON formatted like this, selecting     │
│ when a person has a forgotten memory or experience     │ only the number of the option:                         │
│ that resurfaces without them realizing it's from a     │ {"answer": <put answer code here>, "comment": "<put    │
│ past encounter. As a result, the individual mistakenly │ explanation here>"}                                    │
│ believes that the idea is a new, original thought or   │ Only 1 option may be selected.                         │
│ creation of their own, when in reality, it has been    │         Before the question you are now answering, you │
│ encountered before and merely forgotten.               │ already answered the following question(s):            │
│                                                        │                 Question: Describe the phenomenon of   │
│                                                        │ hearing about an idea and then                         │
│                                                        │     believing that you are responsible for it.         │
│                                                        │         Answer: The phenomenon of hearing about an     │
│                                                        │ idea and then believing that you are responsible for   │
│                                                        │ it is often referred to as 'cryptomnesia.' It occurs   │
│                                                        │ when a person has a forgotten memory or experience     │
│                                                        │ that resurfaces without them realizing it's from a     │
│                                                        │ past encounter. As a result, the individual mistakenly │
│                                                        │ believes that the idea is a new, original thought or   │
│                                                        │ creation of their own, when in reality, it has been    │
│                                                       

In [11]:
results.select("ideas_no_credit_user_prompt", "original_no_credit_user_prompt").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ prompt                                                 ┃ prompt                                                 ┃
┃ .ideas_no_credit_user_prompt                           ┃ .original_no_credit_user_prompt                        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ You are being asked the following question: Give me an │ You are being asked the following question: The idea   │
│ original idea for an applications for large language   │ you just described - is it original?                   │
│ models.                                                │ The options are                                        │
│ Return a valid JSON formatted like this:               │                                                        │
│ {"answer": "<put free text answer here>"}              │ 0: Yes, I thought of it myself.                        │
│                                                        │                                                        │
│                                                        │ 1: I may have heard about it before.                   │
│                                                        │                                                        │
│                                                        │ 2: No, I have heard about it before.                   │
│                                                        │                                                        │
│                                                        │ 3: I do not know.                                      │
│                                                        │                                                        │
│                                                        │ Return a valid JSON formatted like this, selecting     │
│                                                        │ only the number of the option:                         │
│                                                        │ {"answer": <put answer code here>, "comment": "<put    │
│                                                        │ explanation here>"}                                    │
│                                                        │ Only 1 option may be selected.                         │
│                                                        │         Before the question you are now answering, you │
│                                                        │ already answered the following question(s):            │
│                                                        │                 Question: Give me an original idea for │
│                                                        │ an applications for large language models.             │
│                                                        │         Answer: One original application for large     │
│                                                        │ language models could be a 'Cultural Exchange          │
│                                                        │ Companion' app. This platform would use the power of   │
│                                                        │ language models to facilitate cross-cultural           │
│                                                        │ understanding and language learning. Users could input │
│                                                        │ their native language and cultural context, and the    │
│                                                        │ app would pair them with content and conversations     │
│                                                        │ that are tailored to bridge gaps between different     │
│                                                        │ cultures. It would provide real-time translation,      │
│                                                       